* https://uniswapv3book.com/index.html
* https://medium.com/@chaisomsri96/defi-math-uniswap-v3-concentrated-liquidity-bd87686b3ecf
* https://liobaheimbach.github.io/assets/pdf/Papers/Risks_and_Returns_of_Uniswap_V3_Liquidity_Providers.pdf
* https://docs.uniswap.org/sdk/v3/guides/advanced/active-liquidity
* https://blog.uniswap.org/uniswap-v3-math-primer
* https://blog.uniswap.org/uniswap-v3-math-primer-2
* https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

In [1]:
from uniswappy import *
import pandas as pd
import copy
import matplotlib.pyplot as plt

user_nm = MockAddress().apply()
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [2]:
def calc_Lx(sqrtP, dx, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Lx = dx/(1/p_sqrt_human - 1/pb_sqrt_human)
    return Lx

def calc_Ly(sqrtP, dy, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Ly = dy/(p_sqrt_human - pa_sqrt_human)
    return Ly

def calc_y(lp, dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dy = dL*(p_sqrt_human - pa_sqrt_human)
    #y = dL*(pb_sqrt_human - pa_sqrt_human)
    return dy

def calc_x(lp, dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dx = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    #x = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    return dx

In [3]:
q96 = 2**96
eth_gwei = 10**18
def price_to_sqrtp(p):
    return int(math.sqrt(p) * q96)

def tick_to_price(tick):
    return math.floor(1.0001**tick)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

def liquidity0(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)


def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

In [4]:
def calc_lp_settlement(lp, token_in, diy, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    if(token_in.token_name == lp.token0):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
        dPy = (sqrtp_cur - sqrtp_pa)
        dPx = (1/sqrtp_cur - 1/sqrtp_pb)          
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)
        dPx = (1/sqrtp_cur - 1/sqrtp_pa)
        dPy = (sqrtp_cur - sqrtp_pb)

    fee = 997
    
    a = fee*dPy*sqrtp_cur*dPx - 1000*dPx*(sqrtp_cur**2) - fee*dPy  
    b = -fee*dPy*sqrtp_cur*diy + 1000*diy*(sqrtp_cur**2) + L*fee*dPy + 1000*L*dPx*(sqrtp_cur**2)
    c = -1000*L*diy*(sqrtp_cur**2)

    return (-b + math.sqrt(b*b - 4*a*c)) / (2*a)

In [5]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

tick_gap = 1000

lwr_tick = lp.get_tick_price(-1, 10, tick_gap)
upr_tick = lp.get_tick_price(1, 10, tick_gap)

#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 100, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), tick_gap)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), tick_gap)
#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

amount_out = 100
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
#p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)

print(f'lwr_tick {lwr_tick}, cur_tick {price_to_tick(lp.get_price(eth))}, upr_tick {upr_tick}')
lp.get_price(eth)

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.9999999999997, TKN = 10340.759310919504
Gross Liquidity: 66597.92378128914 

lwr_tick 22020, cur_tick 23027, upr_tick 24000


10.000000000000002

In [6]:
dL = 3283.3309207565794
diy = 100
L = lp.get_liquidity()

dPy = calc_y(lp, 1, lwr_tick, upr_tick)
dPx = calc_x(lp, 1, lwr_tick, upr_tick)
dx = dL*dPx
dy = dL*dPy

sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96

L_diff = (L - dL) 
sqrtp_next = sqrtp_cur + (997*dy)/(L_diff*1000) 

dx + L_diff * (1/sqrtp_cur - 1/sqrtp_next)

99.99992253362689

In [7]:
dL = 3283.3309207565794   # tick_gap = 1000
#dL = 6298                 # tick_gap = 500

# Burn portion
dPy = calc_y(lp, 1, lwr_tick, upr_tick)
dPx = calc_x(lp, 1, lwr_tick, upr_tick)
dy = dL*dPy
dx = dL*dPx

# Swap portion
eth_gwei = 10**18
L = lp.get_liquidity()
liq = (L - dL) * eth_gwei
amount_in = dy * eth_gwei
sqrtp_cur = price_to_sqrtp(lp.get_price(eth))

price_diff = ((amount_in - 3*amount_in/1000) * q96) // liq
price_next_calc = sqrtp_cur + price_diff
print("New price:", (price_next_calc / q96) ** 2)
print("New sqrtP:", price_next_calc)
#print("New tick:", price_to_tick((price_next_calc / q96) ** 2))

#amount_in = calc_amount1(liq, price_next_calc, sqrtp_cur)
amount_out = calc_amount0(liq, price_next_calc, sqrtp_cur)

print("USDC in:", amount_in / eth_gwei)
print("ETH out:", amount_out / eth_gwei)

dx_swap = amount_out / eth_gwei

# x pulled out
dx + dx_swap

New price: 10.05083685047008
New sqrtP: 2.5117747796363857e+29
USDC in: 509.8077066357803
ETH out: 50.69912253362368


99.99992253362367

In [8]:
amount_in = dy * eth_gwei
liq = (L - dL) * eth_gwei
price_next_calc = sqrtp_cur + ((amount_in - 3*amount_in/1000) * q96) // liq

dL*dPx + int(liq * q96 * (1/sqrtp_cur - 1/price_next_calc))/ eth_gwei

99.99992253362689

In [9]:
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96

dPx = (1/sqrtp_cur - 1/sqrtp_pb)
dPy = (sqrtp_cur - sqrtp_pa)

In [10]:
diy = 99.99992253362689

In [11]:
L - 997*dy*sqrtp_cur*(diy-dx)/(1000*dx*sqrtp_cur**2 + 997*dy - 1000*diy*sqrtp_cur**2)

3283.330919046348

In [12]:
dx = dL*dPx
dy = dL*dPy

L_diff = (L - dL) 
sqrtp_next = sqrtp_cur + (997*dy)/(L_diff*1000) 

dx + L_diff * (1/sqrtp_cur - 1/sqrtp_next) - diy

0.0

In [13]:
dx + L_diff*(1/sqrtp_cur - 1/(sqrtp_cur + (997*dy)/(L_diff*1000))) - diy

0.0

In [14]:
dx + L_diff*(1/sqrtp_cur - L_diff*1000/(sqrtp_cur*L_diff*1000 + 997*dy)) - diy

0.0

In [15]:
dx + L_diff*( (sqrtp_cur*L_diff*1000 + 997*dy - L_diff*1000*sqrtp_cur)  /  (sqrtp_cur * (sqrtp_cur*L_diff*1000 + 997*dy))  ) - diy

-3.723243935382925e-12

In [16]:
dx + (997*dy*L_diff)  /  (1000*L_diff*sqrtp_cur**2 + 997*dy*sqrtp_cur) - diy

-3.467448550509289e-12

In [17]:
dx*(1000*L_diff*sqrtp_cur**2 + 997*dy*sqrtp_cur) + (997*dy*L_diff) - diy*(1000*L_diff*sqrtp_cur**2 + 997*dy*sqrtp_cur)

-0.00220489501953125

In [18]:
1000*dx*L_diff*sqrtp_cur**2 + 997*dy*dx*sqrtp_cur + 997*dy*L_diff - 1000*L_diff*diy*sqrtp_cur**2 - 997*dy*diy*sqrtp_cur

-0.00221097469329834

In [19]:
(1000*dx*L_diff*sqrtp_cur**2 + 997*dy*L_diff - 1000*L_diff*diy*sqrtp_cur**2)  + (997*dy*dx*sqrtp_cur - 997*dy*diy*sqrtp_cur)

-0.0022064298391342163

In [20]:
L_diff*(1000*dx*sqrtp_cur**2 + 997*dy - 1000*diy*sqrtp_cur**2)  + (997*dy*dx*sqrtp_cur - 997*dy*diy*sqrtp_cur)

-0.002201169729232788

In [21]:
L_diff*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)  + 997*dL*dPy*sqrtp_cur*(dL*dPx-diy)

-0.002208530902862549

In [22]:
(L - dL)*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)  + 997*dL*dPy*sqrtp_cur*(dL*dPx-diy)

-0.002208530902862549

In [23]:
(L - dL)*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)  + 997*dL*dPy*sqrtp_cur*dL*dPx - 997*dL*dPy*sqrtp_cur*diy

-0.002208530902862549

In [24]:
(L - dL)*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)  + 997*dPy*sqrtp_cur*dPx*dL**2 - 997*dL*dPy*sqrtp_cur*diy

-0.002208530902862549

In [25]:
a = L*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)
b = -dL*(1000*dL*dPx*sqrtp_cur**2 + 997*dL*dPy - 1000*diy*sqrtp_cur**2)
d = 997*dPy*sqrtp_cur*dPx*dL**2 
e = - 997*dL*dPy*sqrtp_cur*diy

a + b + d + e

-0.002208530902862549

In [26]:
a = dL*1000*L*dPx*(sqrtp_cur**2) + dL*L*997*dPy - 1000*L*diy*(sqrtp_cur**2)
b = -(dL**2)*1000*dPx*(sqrtp_cur**2) - (dL**2)*997*dPy + dL*1000*diy*(sqrtp_cur**2)
d = (dL**2)*997*dPy*sqrtp_cur*dPx
e = -dL*997*dPy*sqrtp_cur*diy

a + b + d + e

-0.0022050142288208008

In [27]:
a = (dL**2)*997*dPy*sqrtp_cur*dPx  -(dL**2)*1000*dPx*(sqrtp_cur**2) - (dL**2)*997*dPy 
b = -dL*997*dPy*sqrtp_cur*diy + dL*1000*diy*(sqrtp_cur**2) + dL*L*997*dPy + dL*1000*L*dPx*(sqrtp_cur**2) 
c = -1000*L*diy*(sqrtp_cur**2)

a + b + c

-0.002197265625

In [28]:
a = (dL**2)* (997*dPy*sqrtp_cur*dPx - 1000*dPx*(sqrtp_cur**2) - 997*dPy )   
b = dL*(-997*dPy*sqrtp_cur*diy + 1000*diy*(sqrtp_cur**2) + L*997*dPy + 1000*L*dPx*(sqrtp_cur**2) )  
c = -1000*L*diy*(sqrtp_cur**2)

a + b + c

-0.002197265625

In [29]:
a = (dL**2)* (997*dPy*sqrtp_cur*dPx - 1000*dPx*(sqrtp_cur**2) - 997*dPy)   
b = dL*(-997*dPy*sqrtp_cur*diy + 1000*diy*(sqrtp_cur**2) + L*997*dPy + 1000*L*dPx*(sqrtp_cur**2) )  
c = -1000*L*diy*(sqrtp_cur**2)

a + b + c

-0.002197265625

In [30]:
diy = 100

a = (997*dPy*sqrtp_cur*dPx - 1000*dPx*(sqrtp_cur**2) - 997*dPy)  
b = -997*dPy*sqrtp_cur*diy + 1000*diy*(sqrtp_cur**2) + L*997*dPy + 1000*L*dPx*(sqrtp_cur**2)
c = -1000*L*diy*(sqrtp_cur**2)

(-b + math.sqrt(b*b - 4*a*c)) / (2*a)

3283.3334676753425

In [31]:
dL_calc = calc_lp_settlement(lp, eth, 100, lwr_tick, upr_tick)

(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, dL_calc)
lp.summary()

out = Swap().apply(lp, tkn, user_nm, amount1) 
lp.summary()
print(f'amount0 {amount0}, amount1 {amount1}')
out[1]

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 950.699161756784, TKN = 9830.951208819835
Gross Liquidity: 63314.5903136138 

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 899.9999999999997, TKN = 10340.759310919504
Gross Liquidity: 63314.5903136138 

amount0 49.30083824321566, amount1 509.8081020996689


-50.69916175678431

In [32]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

tick_gap = 1000
amount_out = 100

lwr_tick = lp.get_tick_price(-1, 10, tick_gap)
upr_tick = lp.get_tick_price(1, 10, tick_gap)
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 100, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

print(f'---- Uni V3 test ----')
print(f' - Withdraw: {amount_out} ETH')
print(f' - Price ticks: lwr_tick {lwr_tick}, cur_tick {price_to_tick(10)}, upr_tick {upr_tick}\n')

print('---- Initial ----\n')
lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)    #3162.2776601683795
lp.summary()


# Calculate LP settlement
dL_calc = calc_lp_settlement(lp, eth, amount_out, lwr_tick, upr_tick)

# Withdraw
(_, _, _, _, amt0, amt1) = lp.burn(user_nm, lwr_tick, upr_tick, dL_calc)

print(f'---- Withdraw {dL_calc:.6} LP tokens for {amt0:.6} ETH and {amt1:.6} TKN ----\n')
lp.summary()

# Swap
out = Swap().apply(lp, tkn, user_nm, amt1) 

print(f'---- Swap {amt1:.6} TKN for {abs(out[1]):.6} ETH ----\n')
lp.summary()

print(f'---- {amt0:.6} (ETH Burned) + {abs(out[1]):.6} (ETH Swapped) = {(amt0 + abs(out[1])):.6} (ETH Withdrawn) ----\n')

---- Uni V3 test ----
 - Withdraw: 100 ETH
 - Price ticks: lwr_tick -887220, cur_tick 23027, upr_tick 887220

---- Initial ----

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.9999999999999, TKN = 10000.0
Gross Liquidity: 3162.277660168379 

---- Withdraw 162.509 LP tokens for 51.3898 ETH and 513.898 TKN ----

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 948.61016621623, TKN = 9486.101662162302
Gross Liquidity: 2999.7687368341976 

---- Swap 513.898 TKN for 48.6102 ETH ----

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 899.9999999999999, TKN = 10000.0
Gross Liquidity: 2999.7687368341976 

---- 51.3898 (ETH Burned) + 48.6102 (ETH Swapped) = 100.0 (ETH Withdrawn) ----

